# Note for So Far

In [123]:
import scrapy
from scrapy.selector import Selector
from qna_crawler.items import QnaCrawlerItem

from collections import defaultdict
import sys
import datetime
import time
from fake_useragent import UserAgent
import random
import datetime
import re
from dateutil.parser import parse

from selenium import webdriver
from selenium.webdriver.chrome.options import DesiredCapabilities
from selenium.webdriver.common.proxy import Proxy, ProxyType
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver import ActionChains
from selenium.webdriver.common.keys import Keys

#import gspread
#from oauth2client.service_account import ServiceAccountCredentials

# Testing Datetime

Datetime을 사용함에 있어 결국, parse를 사용한 후, Datetime.datetime -> Datetime.date() 변환하는 것이 최선의 선택

In [4]:
today = datetime.date(2020,2,26)
tomorrow = datetime.date(2020,2,27)

In [5]:
datetime.date(2020,2,28)>= tomorrow

True

In [6]:
test = '2020/02/27'.split('/')

In [7]:
datetime.date(int(test[0]), int(test[1]), int(test[2]))

datetime.date(2020, 2, 27)

# Proxies and IP

In [8]:
from selenium import webdriver
from selenium.webdriver.chrome.options import DesiredCapabilities
from selenium.webdriver.common.proxy import Proxy, ProxyType
import time
from fake_useragent import UserAgent
ua = UserAgent()


co = webdriver.ChromeOptions()
co.add_argument("log-level=3")
co.add_argument("--headless")
co.add_argument(f"user-agent={ua.random}")


def get_proxies(co=co):
    driver = webdriver.Chrome(chrome_options=co)
    driver.get("https://free-proxy-list.net/")

    PROXIES = []
    proxies = driver.find_elements_by_css_selector("tr[role='row']")
    for p in proxies:
        result = p.text.split(" ")
        print(result)
        if result[-1] == "yes":
            PROXIES.append(result[0]+":"+result[1])

    driver.close()
    return PROXIES


ALL_PROXIES = get_proxies()

ModuleNotFoundError: No module named 'fake_useragent'

# Functions list

In [39]:
def parse_date(date_value):
    return parse(date_value)

def set_range(start, till):
    start, till = parse(start), parse(till)
    return start, till

def set_range_with_today():
    today = datetime.date.today()
    yesterday = today - datetime.timedelta(days=1)
    day_before_yesterday = yesterday - datetime.timedelta(days=1)
    return yesterday, day_before_yesterday

def inputs_to_dictionary(dictionary, date_value, start_value):
    if date_value == start_value:
        dictionary[date_value] += 1
        return dictionary
    elif date_value > start_value:
        print('out of date-range, over {}'.format(str(start_value)))
        return dictionary
    elif date_value < start_value:
        print('out of date-range, less {}'.format(str(start_value-datetime.timedelta(days=1))))
        return dictionary
    
def set_teachers(teacher_name, doc):
    return doc.worksheet(teacher_name)

def mergeDict(dict1, dict2):
    ''' Merge dictionaries and keep values of common keys in list'''
    dict3 = {**dict1, **dict2}
    for key, value in dict3.items():
        if key in dict1 and key in dict2:
            dict3[key] = value + dict1[key]
 
    return dict3

def check_date_range(date, start, till):
    if date <= start and date >= till:
        return 1
    elif date > start:
        return 0
    elif date < till:
        return -1
    
def row_filter(rows, teacher_name, start, till = None):
    run = True
    
    if till == None:
        till = start
    qna_dic = defaultdict(int)
    start, till = parse_date(start), parse_date(till)
    
    filtered_notice_rows = [row for row in rows if row.get_attribute("class") in [None, '']]
    
    #ETOOS
    if 'etoos' in teacher_name:
        filtered_rows = []
        for row in filtered_notice_rows:
            td_list = row.find_elements_by_css_selector("*")
            
            date_value, writer= parse_date(td_list[-1].text), td_list[-2].text
            if len(writer) > 4:
                print("This is not a question, but answer")
                continue
                
            check = check_date_range(date_value, start, till)
            if check:
                qna_dic[date_value] +=1
            elif check < 0:
                run = False
                break
            
        return run, qna_dic
    
    #SKYEDU
    elif 'skyedu' in teacher_name:
        for row in filtered_notice_rows:
            td_list = row.find_elements_by_css_selector("*")
            date_value = parse_date(td_list[-1].text)
            
            check = check_date_range(date_value, start, till)
            if check:
                qna_dic[date_value] +=1
            elif check < 0:
                run = False
                break

        return run, qna_dic

    #MEGASTUDY
    elif 'megastudy' in teacher_name:
        for row in filtered_notice_rows:
            td_list = row.find_elements_by_css_selector("*")
            date_value = parse_date(td_list[-2].text)
            
            check = check_date_range(date_value, start, till)
            if check:
                qna_dic[date_value] +=1
            elif check < 0:
                run = False
                break
                
        return run, qna_dic
        
    #MIMAC
    elif 'mimac' in teacher_name:
        filtered_rows = []
        for row in filtered_notice_rows:
            td_list = row.find_elements_by_css_selector("*")
            first_td_class = td_list[0].get_attribute("class")
            
            if first_td_class == "noti" or row_class == "reply":
                continue
            date_value = td_list[-2]
            
            check = check_date_range(date_value, start, till)
            if check:
                qna_dic[date_value] +=1
            elif check < 0:
                run = False
                break

        return run, qna_dic
    
def connect_gspread():
    scope = [
    "https://spreadsheets.google.com/feeds",
    "https://www.googleapis.com/auth/drive",
    ]

    json_file_name = "woven-arcadia-269609-12b95dbdd1c3.json"

    credentials = ServiceAccountCredentials.from_json_keyfile_name(json_file_name, scope)
    gc = gspread.authorize(credentials)

    spreadsheet_url = "https://docs.google.com/spreadsheets/d/1YEO-EhcPmtj0r0YJzy-xNF6oVEdb8QHS43Eusck83so/edit#gid=2122150374"

    # 스프레스시트 문서 가져오기
    doc = gc.open_by_url(spreadsheet_url)
    return doc

def driver_setting():
    #ua = UserAgent()
    co = webdriver.ChromeOptions()
    co.add_argument("/home/yoonhoonsang/internet_lecture/chromedriver")
    co.add_argument("log-level=1")
    #co.add_argument("headless")
    #co.add_argument("user-agent={}".format(ua.random))
    co.add_argument("lang=ko_KR")
    return co

def find_rows_of_table(browser):
    tbody = browser.find_element_by_tag_name("tbody")
    rows = tbody.find_elements_by_tag_name("tr")
    return rows

In [51]:
datetime.date.today()

datetime.date(2020, 3, 10)

In [55]:
parse_date('2020-02-02').date()

datetime.date(2020, 2, 2)

In [54]:
datetime.date((parse_date('2020-02-02')))

TypeError: an integer is required (got type datetime.datetime)

In [3]:
def parse_date(date_value):
    return parse(date_value)

In [20]:
def set_range(start, till):
    start, till = parse(start), parse(till)
    return start, till

def set_range_with_today():
    today = datetime.date.today()
    yesterday = today - datetime.timedelta(days=1)
    day_before_yesterday = yesterday - datetime.timedelta(days=1)
    return yesterday, day_before_yesterday

In [6]:
def inputs_to_dictionary(dictionary, date_value, start_value):
    if date_value == start_value:
        dictionary[date_value] += 1
        return dictionary
    elif date_value > start_value:
        print('out of date-range, over {}'.format(str(start_value)))
        return dictionary
    elif date_value < start_value:
        print('out of date-range, less {}'.format(str(start_value-datetime.timedelta(days=1))))
        return dictionary

In [7]:
def set_teachers(teacher_name, doc):
    return doc.worksheet(teacher_name)

In [8]:
def mergeDict(dict1, dict2):
   ''' Merge dictionaries and keep values of common keys in list'''
   dict3 = {**dict1, **dict2}
   for key, value in dict3.items():
       if key in dict1 and key in dict2:
               dict3[key] = value + dict1[key]
 
   return dict3

In [10]:
def check_date_range(date, start, till):
    if date <= start and date >= till:
        return 1
    elif date > start:
        return 0
    elif date < till:
        return -1

In [11]:
check_date_range(parse_date('2020-01-06'), parse_date('2020-01-05'), parse_date('2020-01-01'))

0

In [ ]:
def wait_element(browser, teacher_name):
    if 'etoos' in teacher_name:
        title = WebDriverWait(browser, 10).until(
                EC.presence_of_element_located(
                    (By.CSS_SELECTOR, "table.subcomm_tbl_board")
                )
            )
    elif 'megastudy' in teacher_name:
        title = WebDriverWait(self.browser, 10).until(
                    EC.presence_of_element_located(
                        (
                            By.CSS_SELECTOR,
                            "div.table_list > table.commonBoardList > tbody > tr.top",
                        )
                    )
                )
    elif 'skyedu' in teacher_name:
        title = WebDriverWait(self.browser, 10).until(
                    EC.presence_of_element_located(
                        (By.CSS_SELECTOR, "div.board-list > table")
                    )
                )
    elif 'mimac' in teacher_name:
        title = WebDriverWait(self.browser, 10).until(
                    EC.presence_of_element_located(
                        (By.CSS_SELECTOR, "div.tbltype_list > table")
                    )
                )
        

In [12]:
def row_filter(rows, teacher_name, start, till = None):
    run = True
    
    if till == None:
        till = start
    qna_dic = defaultdict(int)
    start, till = parse_date(start), parse_date(till)
    
    filtered_notice_rows = [row for row in rows if row.get_attribute("class") in [None, '']]
    
    if 'etoos' in teacher_name:
        filtered_rows = []
        for row in filtered_notice_rows:
            td_list = row.find_elements_by_css_selector("*")
            
            date_value, writer= parse_date(td_list[-1].text), td_list[-2].text
            if len(writer) > 4:
                print("This is not a question, but answer")
                continue
                
            check = check_date_range(date_value, start, till)
            if check:
                qna_dic[date_value] +=1
            elif check < 0:
                run = False
                break
            
        return run, qna_dic
    
    
    elif 'skyedu' in teacher_name:
        for row in filtered_notice_rows:
            td_list = row.find_elements_by_css_selector("*")
            date_value = parse_date(td_list[-1].text)
            
            check = check_date_range(date_value, start, till)
            if check:
                qna_dic[date_value] +=1
            elif check < 0:
                run = False
                break

        return run, qna_dic

    
    elif 'megastudy' in teacher_name:
        for row in filtered_notice_rows:
            td_list = row.find_elements_by_css_selector("*")
            date_value = parse_date(td_list[-2].text)
            
            check = check_date_range(date_value, start, till)
            if check:
                qna_dic[date_value] +=1
            elif check < 0:
                run = False
                break
        
    
    elif 'mimac' in teacher_name:
        filtered_rows = []
        for row in filtered_notice_rows:
            td_list = row.find_elements_by_css_selector("*")
            first_td_class = td_list[0].get_attribute("class")
            
            if first_td_class == "noti" or row_class == "reply":
                continue
            date_value = td_list[-2]
            
            check = check_date_range(date_value, start, till)
            if check:
                qna_dic[date_value] +=1
            elif check < 0:
                run = False
                break
            
            

        return filtered_rows
        
    return filtered_notice_rows

In [13]:
def connect_gspread():
    scope = [
    "https://spreadsheets.google.com/feeds",
    "https://www.googleapis.com/auth/drive",
    ]

    json_file_name = "woven-arcadia-269609-12b95dbdd1c3.json"

    credentials = ServiceAccountCredentials.from_json_keyfile_name(json_file_name, scope)
    gc = gspread.authorize(credentials)

    spreadsheet_url = "https://docs.google.com/spreadsheets/d/1YEO-EhcPmtj0r0YJzy-xNF6oVEdb8QHS43Eusck83so/edit#gid=2122150374"

    # 스프레스시트 문서 가져오기
    doc = gc.open_by_url(spreadsheet_url)
    return doc

def driver_setting():
    #ua = UserAgent()
    co = webdriver.ChromeOptions()
    co.add_argument("/home/yoonhoonsang/internet_lecture/chromedriver")
    co.add_argument("log-level=1")
    #co.add_argument("headless")
    #co.add_argument("user-agent={}".format(ua.random))
    co.add_argument("lang=ko_KR")
    return co

def find_rows_of_table(browser):
    tbody = browser.find_element_by_tag_name("tbody")
    rows = tbody.find_elements_by_tag_name("tr")
    return rows

In [14]:
def driver_setting():
    #ua = UserAgent()
    co = webdriver.ChromeOptions()
    co.add_argument("/home/yoonhoonsang/internet_lecture/chromedriver")
    co.add_argument("log-level=1")
    #co.add_argument("headless")
    #co.add_argument("user-agent={}".format(ua.random))
    co.add_argument("lang=ko_KR")
    return co

In [15]:
def find_rows_of_table(browser):
    tbody = browser.find_element_by_tag_name("tbody")
    rows = tbody.find_elements_by_tag_name("tr")
    return rows

In [93]:
teacher_dic = {
    "etoos_yhk": "https://www.etoos.com/teacher/board/sub04_math/board_list.asp?teacher_id=200386&selSearchType=&txtSearchWD=&BOARD_ID=2007&QUST_TYPE_CD=&GOOD_QUST_YN=&MOV_YN=&MEM_YN=&NTView=&page=30",
    "etoos_kww": "https://go3.etoos.com/teacher/board/sub04_math/board_list.asp?teacher_id=200245&selSearchType=&txtSearchWD=&BOARD_ID=2007&QUST_TYPE_CD=&GOOD_QUST_YN=&MOV_YN=&MEM_YN=&NTView=&page=1",
    "megastudy_jjs": "http://www.megastudy.net/teacher_v2/bbs/bbs_list_ax.asp?tec_cd=rimbaud666&tec_nm=%uC870%uC815%uC2DD&tec_type=1&brd_cd=784&brd_tbl=MS_BRD_TEC784&brd_kbn=qnabbs&dom_cd=5&LeftMenuCd=3&LeftSubCd=1&HomeCd=134&page=1&chr_cd=&sub_nm=&ans_yn=&smode=1&sword=&TmpFlg=0.24915805251066403",
    "mimac_lmh": "http://www.mimacstudy.com/tcher/studyQna/getStudyQnaList.ds?tcd=531&currPage=1 ",
    "skyedu_jhc": "https://skyedu.conects.com/teachers/teacher_qna/?t_id=jhc01&cat1=1&page=1",
}

In [16]:
b = webdriver.Chrome(chrome_options=driver_setting())

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: use options instead of chrome_options
  """Entry point for launching an IPython kernel.


In [31]:
b.get(teacher_dic['etoos_yhk'])

In [32]:
rows = find_rows_of_table(b)

In [33]:
rows

[<selenium.webdriver.remote.webelement.WebElement (session="a8a53b82cd1e649b2efea7e4973cda32", element="ae1d6a33-15eb-4e3f-91b7-2abff15980f6")>,
 <selenium.webdriver.remote.webelement.WebElement (session="a8a53b82cd1e649b2efea7e4973cda32", element="b5d8529b-f4ef-4921-ab54-8278f87af520")>,
 <selenium.webdriver.remote.webelement.WebElement (session="a8a53b82cd1e649b2efea7e4973cda32", element="e9395dd8-b992-41f8-8be5-d847632a7f5d")>,
 <selenium.webdriver.remote.webelement.WebElement (session="a8a53b82cd1e649b2efea7e4973cda32", element="0415a1e9-3d2e-49b6-9cb7-b67882063112")>,
 <selenium.webdriver.remote.webelement.WebElement (session="a8a53b82cd1e649b2efea7e4973cda32", element="1e3deca2-532c-4cee-abe3-104ea5b01a49")>,
 <selenium.webdriver.remote.webelement.WebElement (session="a8a53b82cd1e649b2efea7e4973cda32", element="b1fd6378-ecf1-41b2-a3b9-5d33716d8372")>,
 <selenium.webdriver.remote.webelement.WebElement (session="a8a53b82cd1e649b2efea7e4973cda32", element="bc5e8d39-2d69-4689-9e02-f7

In [34]:
row_filter(rows, 'etoos_yhk', '2020-03-10')

This is not a question, but answer
This is not a question, but answer
This is not a question, but answer
This is not a question, but answer
This is not a question, but answer
This is not a question, but answer
This is not a question, but answer
This is not a question, but answer
This is not a question, but answer


(True, defaultdict(int, {datetime.datetime(2020, 3, 8, 0, 0): 11}))

In [268]:
row_filter_by_date(rows, 'etoos_yhk','2020-03-09', '2020-03-08')

TypeError: filter expected 2 arguments, got 4

In [29]:
print(len(rows))
print(len(row_filter(rows, 'etoos_yhk', '2020-03-10')))

23
2


In [36]:
class SPIDER_BOARD(scrapy.Spider):
    name = "SPIDERMAN"
    start_urls = ["http://www.naver.com"]
    
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        # Arguments:
        # self.start, self.till, self.teacher
        # if only self.teacher:   set_range_with_today()
        
        if with_range == False:
            self.start, self.till = set_range_with_today()
        else:
            self.start, self.till = set_range(self.start, self.till)
        self.co = driver_setting()
        self.doc = connect_gspread()
        self.worksheet = set_teachers(self.teacher, doc)
        self.teacher_dic = {
            "etoos_yhk": "https://www.etoos.com/teacher/board/sub04_math/board_list.asp?teacher_id=200386&selSearchType=&txtSearchWD=&BOARD_ID=2007&QUST_TYPE_CD=&GOOD_QUST_YN=&MOV_YN=&MEM_YN=&NTView=&page=10",
            "etoos_kww": "https://go3.etoos.com/teacher/board/sub04_math/board_list.asp?teacher_id=200245&selSearchType=&txtSearchWD=&BOARD_ID=2007&QUST_TYPE_CD=&GOOD_QUST_YN=&MOV_YN=&MEM_YN=&NTView=&page=1",
            "megastudy_jjs": "http://www.megastudy.net/teacher_v2/bbs/bbs_list_ax.asp?tec_cd=rimbaud666&tec_nm=%uC870%uC815%uC2DD&tec_type=1&brd_cd=784&brd_tbl=MS_BRD_TEC784&brd_kbn=qnabbs&dom_cd=5&LeftMenuCd=3&LeftSubCd=1&HomeCd=134&page=1&chr_cd=&sub_nm=&ans_yn=&smode=1&sword=&TmpFlg=0.24915805251066403",
            "mimac_lmh": "http://www.mimacstudy.com/tcher/studyQna/getStudyQnaList.ds?tcd=531&currPage=1 ",
            "skyedu_jhc": "https://skyedu.conects.com/teachers/teacher_qna/?t_id=jhc01&cat1=1&page=1",
        }
        
    def parse(self, response):
        print("---- Scraping Starts ----")
        print("---- Scraping of {}".format(str(self.start)))
        
        page, qna_dictionary, running = 1, defaultdict(int), True
        running = True
        while running:
            self.co.add_argument("user-agent={}".format(ua.random))
            browser = webdriver.Chrome(chrome_options=self.co)
            print("Current Page {}".format(page))
            base_url = self.teacher_dic[self.teacher].format(str(page))
            browser.get(base_url)
            
            print("Accessing {}".format(base_url))
            
            title = WebDriverWait(browser, 10).until(
                EC.presence_of_element_located(
                    (By.CSS_SELECTOR, "table.subcomm_tbl_board")
                )
            )
            print("table element poped up")
            
            rows = find_rows_of_table(browser)
            running, qna_dic = row_filter(rows, self.teacher, self.start, self.till)
            qna_dictionary = mergeDict(qna_dictionary, qna_dic)
            if running == False:
                break
            
        for date in qna_dictionary:
            print("Final Process...")
            self.worksheet.append_row([str(date), str(qna_dictionary[date])])

            item = QnaCrawlerItem()
            item["date"] = date
            item["qna_count"] = qna_dictionary[date]

            yield item

In [102]:
ua = UserAgent()
co = webdriver.ChromeOptions()
co.add_argument("/Users/yoonhoonsang/Desktop/internet_lecture/qna_crawler/chromedriver")
co.add_argument("log-level=1")
co.add_argument("headless")
co.add_argument("user-agent={}".format(ua.random))
co.add_argument("lang=ko_KR")

In [103]:
def get_proxies(co=co):
    driver = webdriver.Chrome(chrome_options=co)
    driver.get("https://free-proxy-list.net/")

    PROXIES = []
    proxies = driver.find_elements_by_css_selector("tr[role='row']")
    for p in proxies:
        result = p.text.split(" ")
        if result[-1] == "yes":
            PROXIES.append(result[0] + ":" + result[1])

    driver.close()
    return PROXIES

In [104]:
ALL_PROXIES = get_proxies()

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:2: DeprecationWarning: use options instead of chrome_options
  


In [105]:
ua = UserAgent()
co = webdriver.ChromeOptions()
co.add_argument("/Users/yoonhoonsang/Desktop/internet_lecture/qna_crawler/chromedriver")
co.add_argument("log-level=1")
co.add_argument("user-agent={}".format(ua.random))
co.add_argument("lang=ko_KR")

In [106]:
ALL_PROXIES

['128.199.77.65:47503',
 '198.50.214.17:8585',
 '190.6.200.158:38256',
 '139.255.123.194:4550',
 '5.202.181.163:8080',
 '195.4.164.127:8080',
 '104.43.211.14:8080']

In [107]:
def proxy_driver(PROXIES, co=co):
    prox = Proxy()
    pxy = ""
    if PROXIES:
        pxy = random.choice(PROXIES) #random proxy
        print(pxy)
    else:
        print("--- Proxies used up (%s)" % len(PROXIES))
        PROXIES = get_proxies()

    prox.proxy_type = ProxyType.MANUAL
    prox.http_proxy = pxy
    prox.ssl_proxy = pxy

    capabilities = webdriver.DesiredCapabilities.CHROME
    prox.add_to_capabilities(capabilities)

    driver = webdriver.Chrome(chrome_options=co, desired_capabilities=capabilities)

    return driver

In [109]:
pd = proxy_driver(ALL_PROXIES)

104.43.211.14:8080


/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:18: DeprecationWarning: use options instead of chrome_options


In [94]:
pd.get("http://www.mimacstudy.com/tcher/studyQna/getStudyQnaList.ds?tcd=531&currPage=1")

In [110]:
import requests

In [113]:
doc = requests.get('http://www.mimacstudy.com/tcher/studyQna/getStudyQnaList.ds?tcd=531&currPage=1')

In [116]:
doc.text

'<!DOCTYPE html>\r\n<html lang="ko">\r\n<head>\r\n<meta http-equiv="X-UA-Compatible" content="IE=edge" />\r\n<!-- <meta property="og:title"\t\tcontent="대성마이맥" /> -->\r\n<!-- <meta property="og:description"\tcontent="전국 고교90%, 서울·경기 고교 99%가 수강! 19패스 하나면 내신부터 수능까지 준비 끝!" /> -->\r\n<!-- <meta property="og:type"\t\tcontent="website" /> -->\r\n<!-- <meta property="og:url"\t\t\tcontent="http://www.mimacstudy.com/" /> -->\r\n<meta http-equiv="Cache-Control" content="no-cache"/>\r\n<meta http-equiv="Expires" content="0"/>\r\n<meta http-equiv="Pragma" content="no-cache"/>\r\n<title>대성마이맥 마이맥선생님 &gt; 학습 Q&A</title>\r\n<meta name="description" content="전국 고교90%, 서울·경기 고교 99%가 수강! 19패스 하나면 내신부터 수능까지 준비 끝!"/>\r\n <link rel="shortcut icon" href="http://img.mimacstudy.com/FRONT/icon/favicon.ico" />\r\n<link rel="stylesheet" type="text/css" href="http://img.mimacstudy.com/FRONT/css/daesung.css?v1">\r\n<link rel="stylesheet" type="text/css" href="http://img.mimacstudy.com/FRONT/css/daesung2.css">\r\n<s

In [118]:
from selenium import webdriver
from selenium.webdriver.firefox.options import Options

options = Options()

In [121]:
driver = webdriver.Firefox(firefox_options=options, executable_path='/usr/local/bin/geckodriver')

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: use options instead of firefox_options
  """Entry point for launching an IPython kernel.


In [122]:
driver.get('http://www.mimacstudy.com/tcher/studyQna/getStudyQnaList.ds?tcd=531&currPage=1')